<a href="https://colab.research.google.com/github/Oleonn/DataMining/blob/Data-mining-Python/PyINaturalist_DataMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyinaturalist

**Importations**

In [8]:
from pyinaturalist.v1.observations import get_observation_species_counts
import json
from pyinaturalist import (
    Observation,
    pprint,
    get_observations,
    get_observation_species_counts
)

import pandas as pd

import csv

import requests

import os

AVANT DE CONTINUER, besoin de fichier "multimedia.txt", exporte de GBIF, dans le repository

In [ ]:
Solcan = pd.read_csv("multimedia.txt", sep="\t")
SolcanTEST = Solcan["identifier"][0:3]


In [ ]:
broken_images = []

for img in SolcanTEST:
    # We can split the file based on '/' and extract the last split within the Python list below:
    file_name = img.split('/')[-2]
    file_name = f"{file_name}.jpeg"  # Update file extension to .jpeg
    print(f"This is the file name: {file_name}")
    # Now let's send a request to the image URL:
    r = requests.get(img, stream=True)
    # We can check that the status code is 200 before doing anything else:
    if r.status_code == 200:
        # This command below will allow us to write the data to a file as binary:
        with open(file_name, 'wb') as f:
            for chunk in r.iter_content(1024):
                f.write(chunk)
    else:
        # We will write all of the images back to the broken_images list:
        broken_images.append(img)
print("Mission accomplished. Good job 007")


This is the file name: 468542.jpeg
This is the file name: 468541.jpeg
This is the file name: 177001.jpeg
Mission accomplished. Good job 007


In [ ]:
for img in SolcanTEST:
    # We can split the file based upon / and extract the last split within the python list below:
    file_name = img.split('/')[-2]
    print(f"This is the file name: {file_name}.jpg")
    # Now let's send a request to the image URL:
    r = requests.get(img, stream=True)
    # We can check that the status code is 200 before doing anything else:
    if r.status_code == 200:
        # This command below will allow us to write the data to a file as binary:
        with open(file_name, 'wb') as f:
            for chunk in r:
                f.write(chunk)
    else:
        # We will write all of the images back to the broken_images list:
        broken_images.append(img)
print("Mission accomplished. Good job 007")

**Classification des resultats en formats utilisables** (desuet)

In [ ]:
Solcan_reponse = get_observations(photos=True, quality="research", taxon_id="67808")
Solcan_datafr = pd.DataFrame.from_dict(Solcan_reponse["results"])
Solcan_photos = Solcan_datafr["observation_photos"]
print(Solcan_reponse)